### 초기설정

In [1]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=ecffe62a3ae4a55fc5acfc3b603a03fe3ecd1f02aa43d8c1d4d3b7e035885653
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [2]:
!pip install umap-learn

     |████████████████████████████████| 80 kB 7.7 MB/s 
     |████████████████████████████████| 1.1 MB 22.0 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-py3-none-any.whl size=76566 sha256=d8250f3a05f00801cebb51544ca647ec10c653093d34ae1fcb0efbb66f5648b5
  Stored in directory: /root/.cache/pip/wheels/01/e7/bb/347dc0e510803d7116a13d592b10cc68262da56a8eec4dd72f
  Created wheel for pynndescent: filename=pynndescent-0.5.4-py3-none-any.whl size=52372 sha256=bc6915822d28b96698479f806a9286021fe978fe25db70ab6015710b1248eda6
  Stored in directory: /root/.cache/pip/wheels/d0/5b/62/3401692ddad12324249c774c4b15ccb046946021e2b581c043
Successfully built umap-learn pynndescent


In [3]:
from efficientnet_pytorch import EfficientNet
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
# from keras.applications.vgg16 import VGG16 
# from keras.models import Model
# from keras.applications.vgg16 import preprocess_input 
import os
import logging
import torch
from torch import nn
import pickle
from tqdm import tqdm
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
#dataset path
data_path = '/content/drive/MyDrive/Colab/Sketch_RNN_Together/Dataset/Filtered_img/'
#feature path
feat_path = "/content/drive/MyDrive/Colab/Sketch_RNN/torch_sketch/features_filter.npy"
#filename path
filename_path = "/content/drive/MyDrive/Colab/Sketch_RNN/torch_sketch/filenames_filter.npy"
#cluster_list
cluster_list_path = '/content/drive/MyDrive/Colab/Sketch_RNN_Together/Dataset/Filtered_img/cluster_list.txt'
cluster_list = open(cluster_list_path , 'r').read().split('\n')
#image_save_path
save_path = '/content/drive/MyDrive/Colab/Sketch_RNN/torch_sketch/Result/'

# p = r"/content/drive/MyDrive/Colab/Sketch_RNN/torch_sketch/sktch_log.pkl"

In [ ]:
import glob
data_list = []
data_list = [files for files in glob.glob(data_path + '*/*.jpg')]
print(data_list)

### Feature Extraction

In [6]:
#transform
tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
pooling = nn.AdaptiveAvgPool2d(1)

def feature_extraction(image, model):
    img = tfms(Image.open(image).convert("RGB")).unsqueeze(0)
    feature = model.extract_features(img)
    features = pooling(feature)
    return features

In [7]:
model = EfficientNet.from_pretrained('efficientnet-b0')
data = {}
for i, image in enumerate(tqdm(data_list)):
  feat = feature_extraction(image, model)
  feat = feat.detach().numpy().reshape(-1)
  data[image] = (feat)
  if(i % 500 == 0):
      np.save(feat_path, np.array(list(data.values())))
      np.save(filename_path, np.array(list(data.keys())))

feat = np.array(list(data.values()))
filenames = np.array(list(data.keys()))
np.save(feat_path, feat)
np.save(filename_path, filenames)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


  0%|          | 0/607 [00:00<?, ?it/s]


Loaded pretrained weights for efficientnet-b0


100%|██████████| 607/607 [05:05<00:00,  1.99it/s]


### Dimension Reduction

In [ ]:
#PCA
# feat = np.load(feat_path)
# from sklearn.decomposition import PCA
# pca = PCA(n_components=2, random_state=0)
# pca.fit(feat)
# x = pca.transform(feat)

In [8]:
#TSNE
feat = np.load(feat_path)
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, init='pca', random_state=0)
x = tsne.fit_transform(feat)

In [ ]:
#UMAP
# feat = np.load(feat_path)
# import umap
# umap = umap.UMAP(n_components=2, random_state=0)
# umap.fit(feat)
# x = umap.transform(feat)

### Clustering

In [16]:
def clustering_method(cmd, cluster_num, input):
  if cmd == 'kmeans':
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=cluster_num, random_state = 0)
    kmeans.fit(input)
    return kmeans.labels_

  elif cmd == 'minibatch':
    from sklearn.cluster import MiniBatchKMeans
    mb_kmeans = MiniBatchKMeans(n_clusters=cluster_num,random_state=0, batch_size=6)
    mb_kmeans.fit(input)
    return mb_kmeans.labels_

  elif cmd == 'dbscan':
    from sklearn.cluster import DBSCAN
    dbscan = DBSCAN(eps=1.3, min_samples=2)
    dbscan.fit(input)
    return dbscan.labels_

  elif cmd == 'birch':
    from sklearn.cluster import Birch
    brc = Birch(threshold=0.3, branching_factor=5, n_clusters=10)
    brc.fit(input)
    return brc.labels_

In [35]:
method = 'birch'
labels = clustering_method(method, len(cluster_list), x)

groups = {}
# holds the cluster id and the images { id: [images] }
for filename, cluster in zip(filenames, labels):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(filename)
    else:
        groups[cluster].append(filename)

In [36]:
#make cluster_dict for calculate acc
cluster_dict = {}
for cluster in groups:
  image_count = []
  image_name = []
  for image in groups[cluster]:
    image_name.append(image.split('/')[-2])
  for name in cluster_list:
    image_count.append(image_name.count(name))

  cluster_dict[cluster] = cluster_list[image_count.index(max(image_count))]

In [37]:
for i in range(len(cluster_dict) - 1):
  print(str(cluster_dict[i]) + ' : ' + str(len(groups[i])))

moustache : 99
cat : 71
cloud : 62
nose : 53
triangle : 73
cat : 51
dog : 69
ear : 20
circle : 64


In [39]:
#Save clustered image
from PIL import Image        
for cluster in groups:
    for image in groups[cluster]:
        path = save_path + '/' + str(cluster) + '_' + str(cluster_dict[cluster]) + '/'
        if os.path.isdir(path) == False:
            os.makedirs(path)
        img = Image.open(image)
        img.save(path + image.split('/')[-1])  

In [40]:
#Acc
from sklearn.metrics import f1_score
pred = []
gt = []    
for cluster in groups:
    for draw in groups[cluster]:
        pred.append(cluster_dict[cluster])
        gt.append(draw.split('/')[-2])

import plotly.express as px
from plotly.offline import plot
fig = px.scatter(x, x=0, y=1, color=labels)
fig.show()
print("F1 ACC: " + str(f1_score(gt, pred, average='micro') * 100))

F1 ACC: 66.88632619439868
